# Generating pitch tracks for the Guše visualizer

In [1]:
import json
import os
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
folderName = '../files/pitchTracks/'
mbid = 'e666047f-d38f-416f-85c6-e345f4286e50'
pitchTrackExtension = '.f0_fil.csv'
fileName = mbid + pitchTrackExtension

dur = 98
shahed = 529.936

filePath = os.path.join(folderName, fileName)
print(filePath)

../files/pitchTracks/e666047f-d38f-416f-85c6-e345f4286e50.f0_fil.csv


### From Hz to cents

In [3]:
def h2c(t, h):
    return np.rint(1200 * np.log2(h/t))

In [4]:
# Load csv
pitchTrackHz = np.genfromtxt(filePath, delimiter=",")
pitchTrackHz

array([[6.00000e-02, 5.21601e+02],
       [7.00000e-02, 5.21612e+02],
       [1.40000e-01, 5.05266e+02],
       ...,
       [9.70700e+01, 3.89690e+02],
       [9.70800e+01, 3.90661e+02],
       [9.70900e+01, 3.93307e+02]])

In [5]:
maxCent = h2c(shahed, np.max(pitchTrackHz[:,1]))
print("Max:", maxCent)

minCent = h2c(shahed, np.min(pitchTrackHz[:,1]))
print("Min:", minCent)

Max: 1024.0
Min: -4758.0


In [6]:
# maxCent = 500
minCent = -2000

minCut = True

In [7]:
# Prepare dic
pitchTrackDic = {}

for frame in pitchTrackHz:
    pitchTrackDic["{:.2f}".format(frame[0])] = frame[1]

In [8]:
# Find duration

if pitchTrackHz[-1,0] > dur:
    dur = pitchTrackHz[-1,0]

dur

98

In [9]:
# Cent pitch track

pitchTrackCents = {}

for i in range(dur * 100):
    k = "{:.2f}".format(i/100)
    if k not in pitchTrackDic.keys():
        pitchTrackCents[k] = "s"
    else:
        c = h2c(shahed, pitchTrackDic[k])
        if minCut:
            if c >= minCent and c <= maxCent:
                pitchTrackCents[k] = c
        else:
            pitchTrackCents[k] = c

pitchTrackCents

{'0.00': 's',
 '0.01': 's',
 '0.02': 's',
 '0.03': 's',
 '0.04': 's',
 '0.05': 's',
 '0.06': -27.0,
 '0.07': -27.0,
 '0.08': 's',
 '0.09': 's',
 '0.10': 's',
 '0.11': 's',
 '0.12': 's',
 '0.13': 's',
 '0.14': -83.0,
 '0.15': -49.0,
 '0.16': -31.0,
 '0.17': -27.0,
 '0.18': -25.0,
 '0.19': -25.0,
 '0.20': -24.0,
 '0.21': -24.0,
 '0.22': -23.0,
 '0.23': -22.0,
 '0.24': -23.0,
 '0.25': -23.0,
 '0.26': -23.0,
 '0.27': -24.0,
 '0.28': -25.0,
 '0.29': -26.0,
 '0.30': -27.0,
 '0.31': -28.0,
 '0.32': -30.0,
 '0.33': -29.0,
 '0.34': -24.0,
 '0.35': -17.0,
 '0.36': -15.0,
 '0.37': -20.0,
 '0.38': -22.0,
 '0.39': -20.0,
 '0.40': -13.0,
 '0.41': 0.0,
 '0.42': 3.0,
 '0.43': 0.0,
 '0.44': 's',
 '0.45': 's',
 '0.46': 's',
 '0.47': 's',
 '0.48': -42.0,
 '0.49': -37.0,
 '0.50': -31.0,
 '0.51': -25.0,
 '0.52': -19.0,
 '0.53': -13.0,
 '0.54': -11.0,
 '0.55': -10.0,
 '0.56': -14.0,
 '0.57': -19.0,
 '0.58': -21.0,
 '0.59': -20.0,
 '0.60': -16.0,
 '0.61': -11.0,
 '0.62': -14.0,
 '0.63': -22.0,
 '0.64': 's',


In [10]:
with open(folderName + mbid + "_pitchTrack.json", 'w') as f:
    json.dump(pitchTrackCents, f)

In [11]:
x = 0
y = 0
for k in pitchTrackCents.keys():
    if pitchTrackCents[k] != 's' and pitchTrackCents[k] > x:
        x = pitchTrackCents[k]
    if pitchTrackCents[k] != 's' and pitchTrackCents[k] < y:
        y = pitchTrackCents[k]
print(y, x)

-1934.0 1024.0


### Pitch histogram

In [12]:
import intonation
import numpy as np
import json
import os

In [13]:
def pitch2cents(pitch, ref):
    '''
    Converts a pitch track in Hz to cents given a reference pitch
    
    Args:
        pitch (np.array): a numpy array with two columns, first for time,
                          second for pitch
        ref (float): the reference pitch
    
    Returns:
        cents (np.array) a numpy array with one column of pitch in cents
    '''

    return 1200 * np.log2(pitch[:,1] / float(ref))

In [14]:
# folderName = './'
# mbid = '7d88c651-5dea-48e5-b21a-ac964cff5cad'
# pitchTrackExtension = '.f0_fil.csv'

# shahed = 332.67

In [15]:
pitchTrack = np.genfromtxt(os.path.join(folderName, mbid + pitchTrackExtension), delimiter=',')
centsTrack = pitch2cents(pitchTrack, shahed)
filteredTrack = centsTrack[np.logical_and(centsTrack >= minCent, centsTrack <= maxCent)]
pitch_obj = intonation.Pitch(np.arange(len(filteredTrack)), filteredTrack)
# pitch_obj = intonation.Pitch(np.arange(len(centsTrack)), centsTrack)
rec_obj = intonation.Recording(pitch_obj)
rec_obj.compute_hist()

print(rec_obj.histogram.x)
print(rec_obj.histogram.y)

[-1933.15770378 -1932.15757104 -1931.1574383  ...  1021.23440607
  1022.23453881  1023.23467155]
[2.30304623e-04 2.31679914e-04 2.34269121e-04 ... 4.92457646e-05
 4.82668321e-05 4.77588732e-05]


In [16]:
histDic = {}

for i in range(rec_obj.histogram.x.size):
    # pos = '{:.2f}'.format(rec_obj.histogram.x[i])
    x = round(rec_obj.histogram.x[i], 2)
    y = round(rec_obj.histogram.y[i] * 10000, 2)
    histDic[x] = y

histMin = np.min(list(histDic.values()))
histMax = np.max(list(histDic.values()))

print(histMin, histMax)

histList = []
for k, v in histDic.items():
    histList.append([k,v])

hist2json = {'min': histMin, 'max': histMax, 'hist': histList}
hist2json

0.0 77.41


{'min': 0.0,
 'max': 77.41,
 'hist': [[-1933.16, 2.3],
  [-1932.16, 2.32],
  [-1931.16, 2.34],
  [-1930.16, 2.38],
  [-1929.16, 2.42],
  [-1928.16, 2.46],
  [-1927.16, 2.49],
  [-1926.16, 2.51],
  [-1925.16, 2.51],
  [-1924.16, 2.5],
  [-1923.16, 2.46],
  [-1922.16, 2.39],
  [-1921.16, 2.29],
  [-1920.16, 2.18],
  [-1919.16, 2.04],
  [-1918.16, 1.88],
  [-1917.16, 1.71],
  [-1916.16, 1.54],
  [-1915.16, 1.36],
  [-1914.16, 1.18],
  [-1913.16, 1.02],
  [-1912.15, 0.86],
  [-1911.15, 0.72],
  [-1910.15, 0.59],
  [-1909.15, 0.48],
  [-1908.15, 0.39],
  [-1907.15, 0.3],
  [-1906.15, 0.24],
  [-1905.15, 0.18],
  [-1904.15, 0.14],
  [-1903.15, 0.1],
  [-1902.15, 0.08],
  [-1901.15, 0.06],
  [-1900.15, 0.04],
  [-1899.15, 0.03],
  [-1898.15, 0.02],
  [-1897.15, 0.01],
  [-1896.15, 0.01],
  [-1895.15, 0.01],
  [-1894.15, 0.0],
  [-1893.15, 0.0],
  [-1892.15, 0.0],
  [-1891.15, 0.0],
  [-1890.15, 0.0],
  [-1889.15, 0.0],
  [-1888.15, 0.0],
  [-1887.15, 0.0],
  [-1886.15, 0.0],
  [-1885.15, 0.0]

In [17]:
pitchHistogramFolder = '../files/pitchHistograms'
pitchHistogramExtension = '_pitchHistogram.json'

json.dumps(hist2json)

with open(os.path.join(pitchHistogramFolder, mbid + pitchHistogramExtension), 'w') as f:
    json.dump(hist2json, f)

In [18]:
minCent

-2000